In [1]:
def selectglaciersrgitable(glac_no=None,
                           rgi_regionsO1=None,
                           rgi_regionsO2=None,
                           rgi_glac_number=None,
#                            rgi_fp=input.rgi_fp,
                           rgi_fp = '/Users/davidrounce/Documents/Dave_Rounce/HiMAT/RGI/rgi60/00_rgi60_attribs/',
                           rgi_cols_drop=['GLIMSId','BgnDate','EndDate','Status','Connect','Linkages','Name'],
                           rgi_O1Id_colname='glacno',
                           rgi_glacno_float_colname='RGIId_float',
                           indexname='GlacNo'):
    """
    Select all glaciers to be used in the model run according to the regions and glacier numbers defined by the RGI
    glacier inventory. This function returns the rgi table associated with all of these glaciers.

    glac_no : list of strings
        list of strings of RGI glacier numbers (e.g., ['1.00001', '13.00001'])
    rgi_regionsO1 : list of integers
        list of integers of RGI order 1 regions (e.g., [1, 13])
    rgi_regionsO2 : list of integers or 'all'
        list of integers of RGI order 2 regions or simply 'all' for all the order 2 regions
    rgi_glac_number : list of strings
        list of RGI glacier numbers without the region (e.g., ['00001', '00002'])

    Output: Pandas DataFrame of the glacier statistics for each glacier in the model run
    (rows = GlacNo, columns = glacier statistics)
    """
    if glac_no is not None:
        glac_no_byregion = {}
        rgi_regionsO1 = [int(i.split('.')[0]) for i in glac_no]
        rgi_regionsO1 = list(set(rgi_regionsO1))
        for region in rgi_regionsO1:
            glac_no_byregion[region] = []
        for i in glac_no:
            region = i.split('.')[0]
            glac_no_only = i.split('.')[1]
            glac_no_byregion[int(region)].append(glac_no_only)

        for region in rgi_regionsO1:
            glac_no_byregion[region] = sorted(glac_no_byregion[region])

    # Create an empty dataframe
    rgi_regionsO1 = sorted(rgi_regionsO1)
    glacier_table = pd.DataFrame()
    for region in rgi_regionsO1:

        if glac_no is not None:
            rgi_glac_number = glac_no_byregion[region]

#        if len(rgi_glac_number) < 50:

        for i in os.listdir(rgi_fp):
            if i.startswith(str(region).zfill(2)) and i.endswith('.csv'):
                rgi_fn = i
        try:
            csv_regionO1 = pd.read_csv(rgi_fp + rgi_fn)
        except:
            csv_regionO1 = pd.read_csv(rgi_fp + rgi_fn, encoding='latin1')
        
        # Populate glacer_table with the glaciers of interest
        if rgi_regionsO2 == 'all' and rgi_glac_number == 'all':
            print("All glaciers within region(s) %s are included in this model run." % (region))
            if glacier_table.empty:
                glacier_table = csv_regionO1
            else:
                glacier_table = pd.concat([glacier_table, csv_regionO1], axis=0)
        elif rgi_regionsO2 != 'all' and rgi_glac_number == 'all':
            print("All glaciers within subregion(s) %s in region %s are included in this model run." %
                  (rgi_regionsO2, region))
            for regionO2 in rgi_regionsO2:
                if glacier_table.empty:
                    glacier_table = csv_regionO1.loc[csv_regionO1['O2Region'] == regionO2]
                else:
                    glacier_table = (pd.concat([glacier_table, csv_regionO1.loc[csv_regionO1['O2Region'] ==
                                                                                regionO2]], axis=0))
        else:
            if len(rgi_glac_number) < 20:
                print("%s glaciers in region %s are included in this model run: %s" % (len(rgi_glac_number), region,
                                                                                       rgi_glac_number))
            else:
                print("%s glaciers in region %s are included in this model run: %s and more" %
                      (len(rgi_glac_number), region, rgi_glac_number[0:50]))
                
            rgiid_subset = ['RGI60-' + str(region).zfill(2) + '.' + x for x in rgi_glac_number] 
            rgiid_all = list(csv_regionO1.RGIId.values)
            rgi_idx = [rgiid_all.index(x) for x in rgiid_subset]
            if glacier_table.empty:
                glacier_table = csv_regionO1.loc[rgi_idx]
            else:
                glacier_table = (pd.concat([glacier_table, csv_regionO1.loc[rgi_idx]],
                                           axis=0))
                    
    glacier_table = glacier_table.copy()
    # reset the index so that it is in sequential order (0, 1, 2, etc.)
    glacier_table.reset_index(inplace=True)
    # change old index to 'O1Index' to be easier to recall what it is
    glacier_table.rename(columns={'index': 'O1Index'}, inplace=True)
    # Record the reference date
    glacier_table['RefDate'] = glacier_table['BgnDate']
    # if there is an end date, then roughly average the year
    enddate_idx = glacier_table.loc[(glacier_table['EndDate'] > 0), 'EndDate'].index.values
    glacier_table.loc[enddate_idx,'RefDate'] = (
            np.mean((glacier_table.loc[enddate_idx,['BgnDate', 'EndDate']].values / 10**4).astype(int),
                    axis=1).astype(int) * 10**4 + 9999)
    # drop columns of data that is not being used
    glacier_table.drop(rgi_cols_drop, axis=1, inplace=True)
    # add column with the O1 glacier numbers
    glacier_table[rgi_O1Id_colname] = (
            glacier_table['RGIId'].str.split('.').apply(pd.Series).loc[:,1].astype(int))
    glacier_table['rgino_str'] = [x.split('-')[1] for x in glacier_table.RGIId.values]
    glacier_table[rgi_glacno_float_colname] = (np.array([np.str.split(glacier_table['RGIId'][x],'-')[1]
                                                    for x in range(glacier_table.shape[0])]).astype(float))
    # set index name
    glacier_table.index.name = indexname

    print("This study is focusing on %s glaciers in region %s" % (glacier_table.shape[0], rgi_regionsO1))

    return glacier_table

In [3]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

# import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib
# from imview.lib import pltlib


import globaldebris_input as input

#INPUT
topdir='/Users/davidrounce/Documents/Dave_Rounce/HiMAT/DEMs/'
#Output directory
outdir = topdir + 'Shean_2019_0213/mb_combined_20190213_nmad_bins/'
outdir_fig = outdir + '/figures/'
outdir_csv = outdir + '/csv'

# #RGI inventory
# glac_str = '15.03473' # Ngozumpa

# roi = 'HMA'

met_sample_fullfn = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/' + 
                     'rounce_model/hma_data/' + roi + '_ERA5-metdata_2000_2018-z.nc')
debris_elevstats_fullfn = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/' +
                           'rounce_model/hma_data/' + roi + '_debris_elevstats.nc')

# glac_shp_fn_dict = {'13':topdir + '../RGI/rgi60/13_rgi60_CentralAsia/13_rgi60_CentralAsia.shp',
#                     '14':topdir + '../RGI/rgi60/14_rgi60_SouthAsiaWest/14_rgi60_SouthAsiaWest.shp',
#                     '15':topdir + '../RGI/rgi60/15_rgi60_SouthAsiaEast/15_rgi60_SouthAsiaEast.shp'}

# glac_shp_fn = glac_shp_fn_dict[region]
# glacfeat_fn = outdir + 'glacfeat_list.p'

In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(outdir):
    if i.endswith('mb_bins.csv'):
        rgiid_list.append(i[0:8])
        rgiid_fn_list.append(i)
        
rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)

print(len(rgiid_list))

main_glac_rgi = selectglaciersrgitable(rgiid_list)
main_glac_rgi['bin_fn'] = rgiid_fn_list

8053
4182 glaciers in region 13 are included in this model run: ['00062', '00093', '00130', '00135', '00137', '00140', '00147', '00175', '00181', '00183', '00203', '00210', '00277', '00358', '00382', '00386', '00391', '00394', '00400', '00401', '00403', '00439', '00440', '00441', '00465', '00561', '00585', '00594', '00604', '00606', '00611', '00628', '00643', '00693', '00713', '00750', '00751', '00757', '00761', '00763', '00777', '00788', '00809', '00830', '00834', '00838', '00880', '00884', '00885', '00891'] and more
2468 glaciers in region 14 are included in this model run: ['00005', '00018', '00032', '00036', '00043', '00057', '00063', '00072', '00088', '00101', '00104', '00111', '00131', '00142', '00145', '00146', '00159', '00163', '00164', '00187', '00213', '00219', '00222', '00225', '00235', '00243', '00251', '00271', '00287', '00309', '00323', '00326', '00336', '00346', '00347', '00352', '00353', '00363', '00366', '00367', '00370', '00372', '00380', '00398', '00403', '00432', '0

In [5]:
print(main_glac_rgi)

        O1Index           RGIId   CenLon   CenLat  O1Region  O2Region    Area  \
GlacNo                                                                          
0            61  RGI60-13.00062  78.1118  35.6407        13         5   2.222   
1            92  RGI60-13.00093  78.0440  35.5883        13         5   2.019   
2           129  RGI60-13.00130  77.9604  35.6074        13         5   2.439   
3           134  RGI60-13.00135  77.9279  35.5521        13         5   7.489   
4           136  RGI60-13.00137  77.9687  35.5987        13         5   2.224   
...         ...             ...      ...      ...       ...       ...     ...   
8048      13091  RGI60-15.13092  97.6170  28.9350        15         3   4.619   
8049      13096  RGI60-15.13097  80.2200  30.5950        15         1   3.616   
8050      13103  RGI60-15.13104  79.4560  31.0520        15         1  22.704   
8051      13115  RGI60-15.13116  81.9910  30.3390        15         1   7.427   
8052      13116  RGI60-15.13

In [6]:
# Group datasets by nearest lat/lon
ds = xr.open_dataset(met_sample_fullfn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi['CenLon'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))

latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))

main_glac_rgi['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi['latlon_unique_no'] = main_glac_rgi['latlon_nearidx'].map(latlon_unique_dict)
print(main_glac_rgi)


        O1Index           RGIId   CenLon   CenLat  O1Region  O2Region    Area  \
GlacNo                                                                          
0            61  RGI60-13.00062  78.1118  35.6407        13         5   2.222   
1            92  RGI60-13.00093  78.0440  35.5883        13         5   2.019   
2           129  RGI60-13.00130  77.9604  35.6074        13         5   2.439   
3           134  RGI60-13.00135  77.9279  35.5521        13         5   7.489   
4           136  RGI60-13.00137  77.9687  35.5987        13         5   2.224   
...         ...             ...      ...      ...       ...       ...     ...   
8048      13091  RGI60-15.13092  97.6170  28.9350        15         3   4.619   
8049      13096  RGI60-15.13097  80.2200  30.5950        15         1   3.616   
8050      13103  RGI60-15.13104  79.4560  31.0520        15         1  22.704   
8051      13115  RGI60-15.13116  81.9910  30.3390        15         1   7.427   
8052      13116  RGI60-15.13

In [29]:
# Process each group and derive elevation statistics for the debris cover
elevstats_mean = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
elevstats_std = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
elevstats_med = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
elevstats_mad = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
elevstats_min = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))
elevstats_max = np.zeros((len(ds.latitude.values), len(ds.longitude.values)))

for nlatlon, latlon_idx in enumerate(list(np.arange(0,len(latlon_nearidx_unique)))):
# for nlatlon, latlon_idx in enumerate([251]):
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['latlon_unique_no'] == latlon_idx]
    bin_fns = main_glac_rgi_subset['bin_fn'].values
    
    lat_idx, lon_idx = latlon_unique_dict_reversed[latlon_idx]
    print(nlatlon, lat_idx, lon_idx)

    df_all = None
    for n, i in enumerate(bin_fns):
        df = pd.read_csv(outdir + i)
        
#         print(i)
        
        # Process only glaciers with debris
        debris_switch=False
        if ' perc_debris' in df.columns:
            # Process dataframe
            output_cns = ['# bin_center_elev_m', ' z1_bin_area_valid_km2', ' perc_debris']
            df = df[output_cns]
            df['# bin_center_elev_m'] = df['# bin_center_elev_m'].astype(np.float) 
            df[' z1_bin_area_valid_km2'] = df[' z1_bin_area_valid_km2'].astype(np.float)
            
            # Remove nan values
            df[' perc_debris'] = df[' perc_debris'].astype(np.float)
            df.fillna(0, inplace=True)
            
#             print(i, df[' perc_debris'].max())
            
            df['area_debris_km2'] = df[' z1_bin_area_valid_km2'] * df[' perc_debris'] / 100
            
            if df[' perc_debris'].max() > 10:
                debris_switch = True
                debris_idx = np.where(df[' perc_debris'] > 50)[0]
                
        if debris_switch:
#             print('processing', i)

            elev_min = df['# bin_center_elev_m'].values[0]
            elev_max = df['# bin_center_elev_m'].values[-1]

            binsize = df['# bin_center_elev_m'].values[1] - df['# bin_center_elev_m'].values[0]

            # Merge datasets together to compute elevation stats for each lat/lon
            if df_all is None:
                df_all = df
                if len(debris_idx) > 1:
                    zmin_debris = df['# bin_center_elev_m'].values[debris_idx[0]]
                    zmax_debris = df['# bin_center_elev_m'].values[debris_idx[-1]]
            else:
                # If new min elevation lower than min old elevation
                if elev_min < df_all['# bin_center_elev_m'].values[0]:
                    elev2add = np.arange(elev_min,df_all['# bin_center_elev_m'].values[0], binsize)
                    df_all_add = pd.DataFrame(np.zeros((len(elev2add), len(df.columns))), columns=df.columns)
                    df_all_add['# bin_center_elev_m'] = elev2add
                    df_all = pd.concat([df_all,df_all_add], axis=0)
                    df_all.sort_values('# bin_center_elev_m', inplace=True)

                # If new max elevation higher than max old elevation
                if elev_max > df_all['# bin_center_elev_m'].values[-1]:
                    elev2add = np.arange(df_all['# bin_center_elev_m'].values[-1] + binsize, elev_max + 1, binsize)
                    df_all_add = pd.DataFrame(np.zeros((len(elev2add), len(df.columns))), columns=df.columns)
                    df_all_add['# bin_center_elev_m'] = elev2add
                    df_all = pd.concat([df_all,df_all_add], axis=0)
                    df_all.sort_values('# bin_center_elev_m', inplace=True)

                # If new min elevation higher than min old elevation
                if df_all['# bin_center_elev_m'].values[0] < elev_min:
                    elev2add = np.arange(df_all['# bin_center_elev_m'].values[0], elev_min, binsize)
                    df_add = pd.DataFrame(np.zeros((len(elev2add), len(df.columns))), columns=df.columns)
                    df_add['# bin_center_elev_m'] = elev2add
                    df = pd.concat([df,df_add], axis=0)
                    df.sort_values('# bin_center_elev_m', inplace=True)

                # If new max elevation lower than max old elevation 
                if df_all['# bin_center_elev_m'].values[-1] > elev_max:
                    elev2add = np.arange(elev_max + binsize, df_all['# bin_center_elev_m'].values[-1] + 1, binsize)
                    df_add = pd.DataFrame(np.zeros((len(elev2add), len(df.columns))), columns=df.columns)
                    df_add['# bin_center_elev_m'] = elev2add
                    df = pd.concat([df,df_add], axis=0)
                    df.sort_values('# bin_center_elev_m', inplace=True)

                df_all.reset_index(inplace=True, drop=True)
                df.reset_index(inplace=True, drop=True)

                # Merge bins (area-weighted)
                df_all[' z1_bin_area_valid_km2'] = df_all[' z1_bin_area_valid_km2'] + df[' z1_bin_area_valid_km2']
                df_all['area_debris_km2'] = df_all['area_debris_km2'] + df['area_debris_km2']
                df_all[' perc_debris'] = df_all['area_debris_km2'] / df_all[' z1_bin_area_valid_km2'] * 100
                
                if len(debris_idx) > 1:
                    if df['# bin_center_elev_m'].values[debris_idx[0]] < zmin_debris:
                        zmin_debris = df['# bin_center_elev_m'].values[debris_idx[0]]
                    if df['# bin_center_elev_m'].values[debris_idx[-1]] > zmax_debris:
                        zmax_debris = df['# bin_center_elev_m'].values[debris_idx[-1]]

    # Area-weighted statistics
    if df_all is not None:
        # Assume 10 m horizontal resolution for computing the area-weighted statistics of the debris elevation
        pixel_res = 10

        # Estimate pixels in each bin
        df_all['pixels_debris'] = np.round(df_all['area_debris_km2'] / (pixel_res / 1000)**2, 0)

        elev_list_all = []
        for nelev, elev in enumerate(df_all['# bin_center_elev_m']):
            elev_list_single = list(np.repeat(elev, df_all.loc[nelev,'pixels_debris']))
            elev_list_all.extend(elev_list_single)

        # Compute statistics
        elev_mean = np.mean(elev_list_all)
        elev_std = np.std(elev_list_all)
        elev_med = malib.fast_median(elev_list_all)
        elev_mad = malib.mad(elev_list_all)

        

        # Update array
        elevstats_mean[lat_idx,lon_idx] = elev_mean
        elevstats_std[lat_idx,lon_idx] = elev_std
        elevstats_med[lat_idx,lon_idx] = elev_med
        elevstats_mad[lat_idx,lon_idx] = elev_mad
        elevstats_min[lat_idx,lon_idx] = zmin_debris
        elevstats_max[lat_idx,lon_idx] = zmax_debris
        
        print('mean +/- std:', np.round(elev_mean,0), '+/-', np.round(elev_std,0), 
              ';  med +/- mad:', np.round(elev_med,0), '+/-', np.round(elev_mad,0), 
              ';  zmin:', zmin_debris, 'zmax:', zmax_debris)

0 0 57
mean +/- std: 3193.0 +/- 141.0 ;  med +/- mad: 3175.0 +/- 148.0 ;  zmin: 2925.0 zmax: 3075.0
1 0 58
mean +/- std: 3291.0 +/- 165.0 ;  med +/- mad: 3275.0 +/- 148.0 ;  zmin: 2875.0 zmax: 3225.0
2 0 59
mean +/- std: 3414.0 +/- 147.0 ;  med +/- mad: 3375.0 +/- 148.0 ;  zmin: 3175.0 zmax: 3275.0
3 0 60
mean +/- std: 3489.0 +/- 234.0 ;  med +/- mad: 3425.0 +/- 222.0 ;  zmin: 3175.0 zmax: 3275.0
4 0 61
mean +/- std: 3390.0 +/- 270.0 ;  med +/- mad: 3325.0 +/- 222.0 ;  zmin: 2975.0 zmax: 3225.0
5 0 62
mean +/- std: 3272.0 +/- 192.0 ;  med +/- mad: 3225.0 +/- 148.0 ;  zmin: 2925.0 zmax: 3125.0
6 0 63
mean +/- std: 3280.0 +/- 143.0 ;  med +/- mad: 3275.0 +/- 148.0 ;  zmin: 2925.0 zmax: 3175.0
7 0 64
mean +/- std: 3406.0 +/- 128.0 ;  med +/- mad: 3375.0 +/- 74.0 ;  zmin: 2925.0 zmax: 3175.0
8 1 59
mean +/- std: 3467.0 +/- 166.0 ;  med +/- mad: 3425.0 +/- 222.0 ;  zmin: 2925.0 zmax: 3175.0
9 1 60
mean +/- std: 3529.0 +/- 185.0 ;  med +/- mad: 3475.0 +/- 148.0 ;  zmin: 3275.0 zmax: 3375.0
1

mean +/- std: 3674.0 +/- 300.0 ;  med +/- mad: 3675.0 +/- 297.0 ;  zmin: 3175.0 zmax: 3975.0
83 10 71
mean +/- std: 3765.0 +/- 216.0 ;  med +/- mad: 3675.0 +/- 148.0 ;  zmin: 3425.0 zmax: 3675.0
84 10 72
mean +/- std: 3929.0 +/- 232.0 ;  med +/- mad: 3875.0 +/- 222.0 ;  zmin: 3625.0 zmax: 3675.0
85 10 73
mean +/- std: 3887.0 +/- 184.0 ;  med +/- mad: 3975.0 +/- 297.0 ;  zmin: 3625.0 zmax: 3675.0
86 10 75
mean +/- std: 3924.0 +/- 211.0 ;  med +/- mad: 3875.0 +/- 222.0 ;  zmin: 3625.0 zmax: 3675.0
87 10 81
mean +/- std: 3963.0 +/- 202.0 ;  med +/- mad: 3925.0 +/- 148.0 ;  zmin: 3625.0 zmax: 4625.0
88 10 82
mean +/- std: 4059.0 +/- 260.0 ;  med +/- mad: 4025.0 +/- 297.0 ;  zmin: 3625.0 zmax: 4625.0
89 11 23
mean +/- std: 3523.0 +/- 160.0 ;  med +/- mad: 3525.0 +/- 222.0 ;  zmin: 3625.0 zmax: 4625.0
90 11 24
mean +/- std: 3704.0 +/- 222.0 ;  med +/- mad: 3725.0 +/- 222.0 ;  zmin: 3625.0 zmax: 4625.0
91 11 25
mean +/- std: 3832.0 +/- 205.0 ;  med +/- mad: 3825.0 +/- 222.0 ;  zmin: 3625.0 zm

mean +/- std: 4151.0 +/- 247.0 ;  med +/- mad: 4075.0 +/- 297.0 ;  zmin: 3725.0 zmax: 4025.0
164 18 38
mean +/- std: 4075.0 +/- 157.0 ;  med +/- mad: 4025.0 +/- 148.0 ;  zmin: 3675.0 zmax: 4025.0
165 18 39
mean +/- std: 4245.0 +/- 120.0 ;  med +/- mad: 4225.0 +/- 74.0 ;  zmin: 3675.0 zmax: 4025.0
166 18 40
mean +/- std: 4214.0 +/- 92.0 ;  med +/- mad: 4225.0 +/- 74.0 ;  zmin: 3675.0 zmax: 4025.0
167 20 30
mean +/- std: 4143.0 +/- 159.0 ;  med +/- mad: 4125.0 +/- 148.0 ;  zmin: 3675.0 zmax: 4025.0
168 21 22
mean +/- std: 3727.0 +/- 180.0 ;  med +/- mad: 3675.0 +/- 222.0 ;  zmin: 3425.0 zmax: 4025.0
169 21 23
mean +/- std: 3441.0 +/- 403.0 ;  med +/- mad: 3425.0 +/- 519.0 ;  zmin: 2775.0 zmax: 3425.0
170 21 25
mean +/- std: 3942.0 +/- 86.0 ;  med +/- mad: 3925.0 +/- 74.0 ;  zmin: 3825.0 zmax: 3975.0
171 21 26
mean +/- std: 3923.0 +/- 198.0 ;  med +/- mad: 3925.0 +/- 148.0 ;  zmin: 3425.0 zmax: 3975.0
172 21 27
mean +/- std: 4056.0 +/- 166.0 ;  med +/- mad: 4025.0 +/- 148.0 ;  zmin: 3575.

mean +/- std: 4182.0 +/- 534.0 ;  med +/- mad: 4175.0 +/- 519.0 ;  zmin: 3075.0 zmax: 4475.0
244 24 122
mean +/- std: 4760.0 +/- 227.0 ;  med +/- mad: 4725.0 +/- 222.0 ;  zmin: 3075.0 zmax: 4475.0
245 24 136
mean +/- std: 4690.0 +/- 56.0 ;  med +/- mad: 4725.0 +/- 74.0 ;  zmin: 3075.0 zmax: 4475.0
246 25 24
mean +/- std: 4029.0 +/- 162.0 ;  med +/- mad: 3975.0 +/- 148.0 ;  zmin: 3075.0 zmax: 4475.0
247 25 25
mean +/- std: 3740.0 +/- 177.0 ;  med +/- mad: 3675.0 +/- 148.0 ;  zmin: 3525.0 zmax: 3975.0
248 25 26
mean +/- std: 3709.0 +/- 317.0 ;  med +/- mad: 3725.0 +/- 371.0 ;  zmin: 3025.0 zmax: 3975.0
249 25 27
mean +/- std: 3835.0 +/- 309.0 ;  med +/- mad: 3775.0 +/- 297.0 ;  zmin: 3025.0 zmax: 3975.0
250 25 28
mean +/- std: 4072.0 +/- 243.0 ;  med +/- mad: 4075.0 +/- 222.0 ;  zmin: 3625.0 zmax: 4125.0
251 25 29
mean +/- std: 3479.0 +/- 584.0 ;  med +/- mad: 3325.0 +/- 445.0 ;  zmin: 2625.0 zmax: 4125.0
252 25 30
mean +/- std: 4621.0 +/- 318.0 ;  med +/- mad: 4625.0 +/- 297.0 ;  zmin: 

mean +/- std: 3955.0 +/- 123.0 ;  med +/- mad: 3925.0 +/- 74.0 ;  zmin: 3775.0 zmax: 3875.0
326 29 25
mean +/- std: 4084.0 +/- 150.0 ;  med +/- mad: 4075.0 +/- 148.0 ;  zmin: 3825.0 zmax: 3925.0
327 29 27
mean +/- std: 4381.0 +/- 184.0 ;  med +/- mad: 4325.0 +/- 148.0 ;  zmin: 3825.0 zmax: 4275.0
328 29 28
mean +/- std: 4338.0 +/- 207.0 ;  med +/- mad: 4275.0 +/- 148.0 ;  zmin: 4025.0 zmax: 4175.0
329 29 29
mean +/- std: 4596.0 +/- 220.0 ;  med +/- mad: 4575.0 +/- 222.0 ;  zmin: 4175.0 zmax: 4375.0
330 29 30
mean +/- std: 4653.0 +/- 175.0 ;  med +/- mad: 4625.0 +/- 148.0 ;  zmin: 4175.0 zmax: 4375.0
331 29 31
mean +/- std: 4754.0 +/- 167.0 ;  med +/- mad: 4725.0 +/- 148.0 ;  zmin: 4175.0 zmax: 4575.0
332 29 93
mean +/- std: 5113.0 +/- 186.0 ;  med +/- mad: 5075.0 +/- 148.0 ;  zmin: 4175.0 zmax: 4575.0
333 29 97
334 29 102
mean +/- std: 5118.0 +/- 203.0 ;  med +/- mad: 5075.0 +/- 148.0 ;  zmin: 4175.0 zmax: 4575.0
335 29 103
mean +/- std: 5171.0 +/- 232.0 ;  med +/- mad: 5025.0 +/- 148.

...

mean +/- std: 4456.0 +/- 285.0 ;  med +/- mad: 4375.0 +/- 297.0 ;  zmin: 3825.0 zmax: 4475.0
837 50 50
mean +/- std: 4773.0 +/- 336.0 ;  med +/- mad: 4775.0 +/- 445.0 ;  zmin: 3825.0 zmax: 5075.0
838 50 51
mean +/- std: 5309.0 +/- 199.0 ;  med +/- mad: 5275.0 +/- 222.0 ;  zmin: 3825.0 zmax: 5125.0
839 50 52
mean +/- std: 5490.0 +/- 169.0 ;  med +/- mad: 5525.0 +/- 148.0 ;  zmin: 3825.0 zmax: 5225.0
840 50 53
mean +/- std: 5508.0 +/- 224.0 ;  med +/- mad: 5475.0 +/- 222.0 ;  zmin: 3825.0 zmax: 5225.0
841 50 54
mean +/- std: 5584.0 +/- 223.0 ;  med +/- mad: 5525.0 +/- 148.0 ;  zmin: 5375.0 zmax: 5425.0
842 50 55
mean +/- std: 5450.0 +/- 124.0 ;  med +/- mad: 5425.0 +/- 148.0 ;  zmin: 5275.0 zmax: 5325.0
843 50 56
mean +/- std: 5617.0 +/- 123.0 ;  med +/- mad: 5625.0 +/- 148.0 ;  zmin: 5275.0 zmax: 5475.0
844 50 90
mean +/- std: 5702.0 +/- 155.0 ;  med +/- mad: 5675.0 +/- 74.0 ;  zmin: 5275.0 zmax: 5475.0
845 50 92
846 50 112
mean +/- std: 5455.0 +/- 192.0 ;  med +/- mad: 5525.0 +/- 222.0

mean +/- std: 5941.0 +/- 212.0 ;  med +/- mad: 5925.0 +/- 222.0 ;  zmin: 3825.0 zmax: 4975.0
919 57 94
mean +/- std: 5709.0 +/- 149.0 ;  med +/- mad: 5725.0 +/- 222.0 ;  zmin: 3825.0 zmax: 4975.0
920 57 95
mean +/- std: 5803.0 +/- 204.0 ;  med +/- mad: 5825.0 +/- 222.0 ;  zmin: 3825.0 zmax: 4975.0
921 57 106
mean +/- std: 5565.0 +/- 199.0 ;  med +/- mad: 5525.0 +/- 222.0 ;  zmin: 5225.0 zmax: 6125.0
922 57 114
mean +/- std: 5334.0 +/- 150.0 ;  med +/- mad: 5325.0 +/- 148.0 ;  zmin: 5075.0 zmax: 5125.0
923 57 115
mean +/- std: 5397.0 +/- 118.0 ;  med +/- mad: 5375.0 +/- 148.0 ;  zmin: 5075.0 zmax: 5125.0
924 57 116
mean +/- std: 5188.0 +/- 329.0 ;  med +/- mad: 5125.0 +/- 297.0 ;  zmin: 4675.0 zmax: 5025.0
925 57 117
mean +/- std: 4924.0 +/- 474.0 ;  med +/- mad: 4925.0 +/- 593.0 ;  zmin: 4125.0 zmax: 4375.0
926 57 118
mean +/- std: 5118.0 +/- 248.0 ;  med +/- mad: 5075.0 +/- 222.0 ;  zmin: 4625.0 zmax: 4925.0
927 57 119
mean +/- std: 4735.0 +/- 367.0 ;  med +/- mad: 4625.0 +/- 222.0 ; 

mean +/- std: 4707.0 +/- 267.0 ;  med +/- mad: 4625.0 +/- 222.0 ;  zmin: 4425.0 zmax: 4725.0
1002 60 119
mean +/- std: 4557.0 +/- 275.0 ;  med +/- mad: 4475.0 +/- 148.0 ;  zmin: 4275.0 zmax: 4425.0
1003 60 120
mean +/- std: 4270.0 +/- 727.0 ;  med +/- mad: 4075.0 +/- 519.0 ;  zmin: 3075.0 zmax: 5725.0
1004 60 121
mean +/- std: 4498.0 +/- 346.0 ;  med +/- mad: 4425.0 +/- 371.0 ;  zmin: 3925.0 zmax: 4275.0
1005 60 122
mean +/- std: 3783.0 +/- 320.0 ;  med +/- mad: 3725.0 +/- 297.0 ;  zmin: 3225.0 zmax: 4525.0
1006 60 123
mean +/- std: 4777.0 +/- 350.0 ;  med +/- mad: 4725.0 +/- 297.0 ;  zmin: 3775.0 zmax: 5525.0
1007 60 124
mean +/- std: 4721.0 +/- 413.0 ;  med +/- mad: 4675.0 +/- 371.0 ;  zmin: 3975.0 zmax: 5825.0
1008 60 125
mean +/- std: 5093.0 +/- 151.0 ;  med +/- mad: 5125.0 +/- 148.0 ;  zmin: 3975.0 zmax: 5825.0
1009 61 64
mean +/- std: 4748.0 +/- 223.0 ;  med +/- mad: 4725.0 +/- 148.0 ;  zmin: 4375.0 zmax: 4875.0
1010 61 66
mean +/- std: 4355.0 +/- 320.0 ;  med +/- mad: 4275.0 +/-

mean +/- std: 5239.0 +/- 361.0 ;  med +/- mad: 5125.0 +/- 222.0 ;  zmin: 4725.0 zmax: 5225.0
1085 65 101
mean +/- std: 5545.0 +/- 152.0 ;  med +/- mad: 5525.0 +/- 74.0 ;  zmin: 4725.0 zmax: 5225.0
1086 65 113
mean +/- std: 5027.0 +/- 134.0 ;  med +/- mad: 5025.0 +/- 74.0 ;  zmin: 4725.0 zmax: 5225.0
1087 65 114
mean +/- std: 4726.0 +/- 237.0 ;  med +/- mad: 4725.0 +/- 297.0 ;  zmin: 4175.0 zmax: 5025.0
1088 65 126
mean +/- std: 4874.0 +/- 249.0 ;  med +/- mad: 4925.0 +/- 297.0 ;  zmin: 4425.0 zmax: 5225.0
1089 65 131
mean +/- std: 4959.0 +/- 204.0 ;  med +/- mad: 4925.0 +/- 222.0 ;  zmin: 4575.0 zmax: 5375.0
1090 65 133
mean +/- std: 4841.0 +/- 178.0 ;  med +/- mad: 4825.0 +/- 222.0 ;  zmin: 4475.0 zmax: 4825.0
1091 66 75
mean +/- std: 4487.0 +/- 448.0 ;  med +/- mad: 4275.0 +/- 371.0 ;  zmin: 3825.0 zmax: 4775.0
1092 66 76
mean +/- std: 4843.0 +/- 322.0 ;  med +/- mad: 4875.0 +/- 371.0 ;  zmin: 3975.0 zmax: 5025.0
1093 66 77
mean +/- std: 4527.0 +/- 386.0 ;  med +/- mad: 4525.0 +/- 29

mean +/- std: 4993.0 +/- 411.0 ;  med +/- mad: 4975.0 +/- 371.0 ;  zmin: 3725.0 zmax: 5225.0
1166 69 95
mean +/- std: 5037.0 +/- 196.0 ;  med +/- mad: 5075.0 +/- 222.0 ;  zmin: 3725.0 zmax: 5225.0
1167 69 96
mean +/- std: 5349.0 +/- 236.0 ;  med +/- mad: 5225.0 +/- 148.0 ;  zmin: 3725.0 zmax: 5225.0
1168 69 97
mean +/- std: 4745.0 +/- 386.0 ;  med +/- mad: 4625.0 +/- 297.0 ;  zmin: 4175.0 zmax: 4975.0
1169 69 108
mean +/- std: 5008.0 +/- 66.0 ;  med +/- mad: 4975.0 +/- 74.0 ;  zmin: 4175.0 zmax: 4975.0
1170 69 109
mean +/- std: 5252.0 +/- 186.0 ;  med +/- mad: 5225.0 +/- 148.0 ;  zmin: 4875.0 zmax: 5275.0
1171 69 110
mean +/- std: 5280.0 +/- 231.0 ;  med +/- mad: 5275.0 +/- 148.0 ;  zmin: 4125.0 zmax: 5275.0
1172 70 92
mean +/- std: 4884.0 +/- 202.0 ;  med +/- mad: 4825.0 +/- 222.0 ;  zmin: 4575.0 zmax: 5125.0
1173 70 93
mean +/- std: 4711.0 +/- 187.0 ;  med +/- mad: 4675.0 +/- 148.0 ;  zmin: 4375.0 zmax: 4875.0


In [30]:
print(df)

    # bin_center_elev_m   z1_bin_area_valid_km2   perc_debris  area_debris_km2
0                4375.0                   0.000          0.00         0.000000
1                4425.0                   0.000          0.00         0.000000
2                4475.0                   0.010        100.00         0.010000
3                4525.0                   0.225         98.54         0.221715
4                4575.0                   0.291         95.70         0.278487
5                4625.0                   0.255         99.23         0.253036
6                4675.0                   0.243        100.00         0.243000
7                4725.0                   0.186         99.65         0.185349
8                4775.0                   0.100         70.78         0.070780
9                4825.0                   0.052         25.97         0.013504
10               4875.0                   0.070         25.47         0.017829
11               4925.0                   0.072     

In [34]:
# Export to dataset
ds_elevstats = xr.Dataset({'zmean': (['latitude', 'longitude'], elevstats_mean),
                           'zstd': (['latitude', 'longitude'], elevstats_std),
                           'zmed': (['latitude', 'longitude'], elevstats_med),
                           'zmad': (['latitude', 'longitude'], elevstats_mad),
                           'zmin': (['latitude', 'longitude'], elevstats_min),
                           'zmax': (['latitude', 'longitude'], elevstats_max),},
                          coords={'latitude': ds.latitude.values,
                                  'longitude': ds.longitude.values})
attrs_dict={
     'zmean':{'units':'m a.s.l.',
         'long_name':'mean elevation',
         'comment': 'mean elevation associated with the debris for the given lat/lon'},
     'zstd':{'units':'m a.s.l.',
         'long_name':'standard deviation of the debris elevation',
         'comment': 'standard deviation of the debris elevation associated with the debris for the given lat/lon'},
     'zmed':{'units':'m a.s.l.',
         'long_name':'median elevation',
         'comment': 'median elevation associated with the debris for the given lat/lon'},
     'zmad':{'units':'m a.s.l.',
         'long_name':'median absolute deviation of the debris elevation',
         'comment': 'median absolute deviation of the debris elevation associated with the debris for the given lat/lon'},
     'zmin':{'units':'m a.s.l.',
         'long_name':'minimum elevation',
         'comment': 'minimum elevation with >50% debris cover for the given lat/lon'},
     'zmax':{'units':'m a.s.l.',
         'long_name':'maximum elevation',
         'comment': 'maximum elevation with >50% debris cover for the given lat/lon'}}

for vn in ['zmean', 'zstd', 'zmed', 'zmad']:
    ds_elevstats[vn].attrs = attrs_dict[vn]
    
ds_elevstats.to_netcdf(debris_elevstats_fullfn.replace('.nc','v2.nc'))
                
print(ds_elevstats)

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 161)
Coordinates:
  * latitude   (latitude) float32 45.0 44.75 44.5 44.25 ... 25.5 25.25 25.0
  * longitude  (longitude) float32 65.0 65.25 65.5 65.75 ... 104.5 104.75 105.0
Data variables:
    zmean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    zstd       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    zmed       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    zmad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    zmin       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    zmax       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0


In [35]:
# lat_idx = 68
# lon_idx = 88
lat_idx = 37
lon_idx = 46
print(ds['latitude'][lat_idx].values, ds['longitude'][lon_idx].values,
      '\n', ds_elevstats['zmean'][lat_idx,lon_idx].values, ds_elevstats['zstd'][lat_idx,lon_idx].values, 
      ds_elevstats['zmed'][lat_idx,lon_idx].values, ds_elevstats['zmad'][lat_idx,lon_idx].values)

35.75 76.5 
 4344.953214642455 413.2688619987504 4325.0 370.65


In [36]:
latlon_unique_dict[(68,88)]

1137